In [7]:
import pandas as pd
import numpy as np
import math as m
from scipy import stats
from statsmodels.stats.diagnostic import acorr_ljungbox
import matplotlib.pyplot as plt
from matplotlib import style
import  seaborn as sns

plt.style.use('ggplot')

In [8]:
#Chargement du DataSet
res=pd.read_excel("histo_cotation1_2017_2018_2019.xlsx")
res.head()

,SEANCE,GROUPE,CODE,VALEUR,OUVERTURE,CLOTURE,PLUS_BAS,PLUS_HAUT,QUANTITE_NEGOCIEE,NB_TRANSACTION,CAPITAUX,IND_RES
0,--------,----------,------------------------,------------------,----------,----------,----------,----------,-----------------,----------,----------,-
1,2019-01-02 00:00:00,11,725001,ADWYA,4.8,4.8,4.8,4.8,4296,5,20620.8,NaN
2,2019-01-02 00:00:00,11,340005,AMEN BANK,27.8,27.75,27.5,27.8,261,9,7240.06,NaN
3,2019-01-02 00:00:00,11,730001,ARTES,6.27,6.28,6.27,6.29,2532,4,15924.1,NaN
4,2019-01-02 00:00:00,11,714001,ASSAD,7.79,8.02,7.78,8.02,14878,62,118592,NaN


In [9]:
BIAT=res[res['VALEUR']=="BIAT"]

In [10]:
BIAT.head()

,SEANCE,GROUPE,CODE,VALEUR,OUVERTURE,CLOTURE,PLUS_BAS,PLUS_HAUT,QUANTITE_NEGOCIEE,NB_TRANSACTION,CAPITAUX,IND_RES
11,2019-01-02 00:00:00,11,180045,BIAT,119.49,118,117,119.49,709,24,83568.9,NaN
318,2019-01-03 00:00:00,11,180045,BIAT,120,117.85,117,120,1230,46,145194,NaN
625,2019-01-04 00:00:00,11,180045,BIAT,117.99,117,116.11,117.99,942,38,110250,NaN
932,2019-01-07 00:00:00,11,180045,BIAT,117,114.01,114.01,117,1612,48,187612,NaN
1239,2019-01-08 00:00:00,11,180045,BIAT,117.3,115,114.01,117.3,1435,37,164601,NaN


In [11]:
B=BIAT[["SEANCE","CLOTURE"]]
B

,SEANCE,CLOTURE
11,2019-01-02 00:00:00,118
318,2019-01-03 00:00:00,117.85
625,2019-01-04 00:00:00,117
932,2019-01-07 00:00:00,114.01
1239,2019-01-08 00:00:00,115
...,...,...
76029,2019-12-25 00:00:00,114.95
76343,2019-12-26 00:00:00,113.2
76657,2019-12-27 00:00:00,113.2
76971,2019-12-30 00:00:00,113.31


In [12]:
#B['dividendes']=0.0
B=B.assign(dividendes=0.0)

In [13]:
#B['rendement']=0.0
B=B.assign(rendement=0.0)

In [14]:
B.head()

,SEANCE,CLOTURE,dividendes,rendement
11,2019-01-02 00:00:00,118,0.0,0.0
318,2019-01-03 00:00:00,117.85,0.0,0.0
625,2019-01-04 00:00:00,117,0.0,0.0
932,2019-01-07 00:00:00,114.01,0.0,0.0
1239,2019-01-08 00:00:00,115,0.0,0.0


In [15]:
B.loc[B.SEANCE == '5/14/2018','dividendes']=4.250
B.loc[B.SEANCE == '5/10/2019','dividendes']=5000

In [16]:
B.head()

,SEANCE,CLOTURE,dividendes,rendement
11,2019-01-02 00:00:00,118,0.0,0.0
318,2019-01-03 00:00:00,117.85,0.0,0.0
625,2019-01-04 00:00:00,117,0.0,0.0
932,2019-01-07 00:00:00,114.01,0.0,0.0
1239,2019-01-08 00:00:00,115,0.0,0.0


In [17]:
B["CLOTURE"].isnull().sum()

0

In [21]:
B.dtypes

SEANCE         object
CLOTURE        object
dividendes    float64
rendement     float64
dtype: object

In [22]:
B['CLOTURE'] = B['CLOTURE'].astype(np.float64)

In [23]:
B.dtypes

SEANCE         object
CLOTURE       float64
dividendes    float64
rendement     float64
dtype: object

In [24]:
#B['dividendes']=p.to_numeric(B['dividendes'],errors='coerce')
#B['CLOTURE']=p.to_numeric(B['CLOTURE'],errors='coerce')
#x= np.log(((B['CLOTURE']+B['dividendes'])/B["CLOTURE"].shift(1)))


In [26]:
x=B['CLOTURE']+B['dividendes']
B['rendement']=np.log((x/B["CLOTURE"].shift(1)))

In [27]:
#Test de normalité
B1=B
B1.rendement=B.rendement.replace([np.inf, -np.inf],0)

In [28]:
B2=B1
B2.rendement=B1.rendement.replace(np.nan,0)

In [29]:
#le test de normalité Shapiro
stats.shapiro(B2.rendement)

(0.9490256905555725, 1.2001829929886298e-07)

In [32]:
#Test de Jarque Bera
stats.jarque_bera(B2.rendement)


(37.9232524082157, 5.821975457287465e-09)

In [34]:
#V.	Calcul de l’option
rc=np.mean(B2.rendement)
rc

-0.00013849869908100717

In [36]:
s=np.sum((B2.rendement-rc)**2)*(255/(len(B2)-1))
s

0.026971689016732994

In [38]:
u=0.5*s+(255*rc)
u

-0.021831323757290327

In [41]:
K=130
S=122
d1=(np.log(S/K) + (0.05 + s/2.0))/(np.sqrt(s))
d1

-0.0001678202140749676

In [42]:
d2=d1-np.sqrt(s)
d2

-0.16439841729249088

In [46]:
C1=122*stats.norm.cdf(d1)-K*np.exp(-0.05)*stats.norm.cdf(d2)
C1

7.235823414242333

In [47]:
st=122*np.exp((0.05-0.5*s)+np.sqrt(s)*x)
st

11       3.299908e+10
318      3.219610e+10
625      2.800125e+10
932      1.713631e+10
1239     2.016177e+10
             ...     
76029    1.999688e+10
76343    1.500184e+10
76657    1.500184e+10
76971    1.527532e+10
77285    1.710819e+10
Length: 249, dtype: float64

In [48]:
a=[]
for i in st:
    a append(max (i-k,0)*np.exp(-0.05))



SyntaxError: invalid syntax (<ipython-input-48-15e2e7c20f2d>, line 3)